In [2]:
from nltk.test.gensim_fixt import setup_module
setup_module()

In [3]:
import gensim

In [24]:
# import nltk
# nltk.download('brown')
from nltk.corpus import brown
train_set = brown.sents()#[:10000]

In [26]:
model = gensim.models.Word2Vec(train_set, negative=0, sg=0, hs=1, cbow_mean=0)

In [27]:
len(model.wv['university'])

100

In [28]:
with open("../dataset/word-test.v1.txt", "r", encoding="utf-8") as f:
    word_test = f.readlines()

In [29]:
def in_top5(x1, x2, y1, y2):
    if x1 not in model.wv.key_to_index or x2 not in model.wv.key_to_index or y1 not in model.wv.key_to_index or y2 not in model.wv.key_to_index:
        return False
    result = model.wv.most_similar(positive=[y1, x2], negative=[x1], topn=5)
    return y2 in list(zip(*result))[0]

count = 0
total = 0
for line in word_test:
    line = line.strip()

    if len(line) == 0 or line.startswith("//"):
        continue
    elif line.startswith(": "):
        print(count, total)
        print(line)
        count = 0
        total  = 0
    else:
        line = line.split()
        if in_top5(line[0], line[1], line[2], line[3]):
            count += 1
        total += 1


0 0
: capital-common-countries
4 506
: capital-world
3 4524
: currency
0 866
: city-in-state
7 2467
: family
74 506
: gram1-adjective-to-adverb
5 992
: gram2-opposite
3 812
: gram3-comparative
202 1332
: gram4-superlative
1 1122
: gram5-present-participle
38 1056
: gram6-nationality-adjective
0 1599
: gram7-past-tense
123 1560
: gram8-plural
29 1332
: gram9-plural-verbs


In [59]:
print(len(model.wv.key_to_index))

15173


In [57]:
import numpy as np

norms = (model.wv.vectors**2).sum(axis=1, keepdims=True) ** (1 / 2)
embeddings_norm = model.wv.vectors / norms

def vector_op(a, b, c, d, show=False, do_norm=True):

    def norm(emb):
        emb_norm = (emb**2).sum() ** (1 / 2)
        emb = emb / emb_norm
        return emb
    
    if a not in model.wv.key_to_index or b not in model.wv.key_to_index or c not in model.wv.key_to_index or d not in model.wv.key_to_index:
        return False


    emb1 = norm(model.wv[a]) if do_norm else model.wv[a]
    emb2 = norm(model.wv[b]) if do_norm else model.wv[b]
    emb3 = norm(model.wv[c]) if do_norm else model.wv[c]

    emb4 = emb2 - emb1 + emb3
    if do_norm:
        emb4 = norm(emb4)

    emb4 = np.reshape(emb4, (len(emb4), 1))
    dists = np.matmul(embeddings_norm if do_norm else model.wv.vectors, emb4).flatten()

    top5 = np.argsort(-dists)[:5]
    ret = model.wv.key_to_index[d] in top5

    if show:
        print(f"{b} - {a} + {c} = ")
        for word_id in top5:
            print("{}: {}".format(model.wv.index_to_key[word_id.item()], dists[word_id]))    

    # top5 = np.argsort(-dists)[:10]
    # for word_id in top5:
    #     print("{}: {:.3f}".format(vocab["i2w"][word_id.item()], dists[word_id]))
    return ret

In [48]:
count = 0
total = 0
for line in word_test:
    line = line.strip()

    if len(line) == 0 or line.startswith("//"):
        continue
    elif line.startswith(": "):
        print(count, total)
        print(line)
        count = 0
        total = 0
    else:
        line = line.split()
        if vector_op(line[0], line[1], line[2], line[3]):
            count += 1
        total += 1

0 0
: capital-common-countries
0 506
: capital-world
0 4524
: currency
0 866
: city-in-state
0 2467
: family
50 506
: gram1-adjective-to-adverb
0 992
: gram2-opposite
0 812
: gram3-comparative
22 1332
: gram4-superlative
0 1122
: gram5-present-participle
10 1056
: gram6-nationality-adjective
0 1599
: gram7-past-tense
133 1560
: gram8-plural
23 1332
: gram9-plural-verbs


In [58]:
vector_op("man", "woman", "king", "queen", show=True, do_norm=False)
print()
vector_op("man", "woman", "king", "queen", show=True, do_norm=True)
print()
print(model.wv.most_similar(positive=["king", "woman"], negative=["man"], topn=5))

woman - man + king = 
per: 0.5599241256713867
Rhode: 0.4594147503376007
St.: 0.3398160934448242
available: 0.2923399806022644
?: 0.2909367084503174

woman - man + king = 
king: 0.794495165348053
ankles: 0.49213099479675293
thoughts: 0.4885934591293335
peers: 0.4820511043071747
submerged: 0.47322896122932434

[('ankles', 0.49213099479675293), ('thoughts', 0.4885934293270111), ('peers', 0.4820511043071747), ('submerged', 0.47322893142700195), ('destination', 0.46488702297210693)]
